# Équipe 7
## Notebook de Gabriel
### Processing des données

In [32]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import datetime as dt


In [33]:
def load_distinction_data():
    '''
    Cette fonction importe les données des distinctions
    de divers individus au courant des années.
    '''
    df = pd.read_csv("../Src/Assets/Data/distinctions_people.csv")

    # Enlever ligne avec erreur
    df.drop([1176], inplace = True)
    
    # Enlever colonnes de liens https
    df.drop(labels = ['person', 'distinction'], axis = 1, inplace = True)

    # Renommer deux colonnes
    df.rename(columns={'personLabel': 'person', 'distinctionLabel': 'distinction'}, inplace  =True)

    # Convertir Format Date en Entier
    df["date"] = df["date"].str[:4].astype('int32')
    
    return df


In [55]:
def backend_load_distinction():
    # Set file path and name
    relative_path = "../Src/Assets/Data/Jc/"
    file_name = "distinction.csv"
    full_path = relative_path + file_name

    # Read .csv
    temp_df = pd.read_csv(
        filepath_or_buffer=full_path,
        sep=',',
        header=0,
        infer_datetime_format=True,
    )

    return temp_df

In [229]:
def clean_distinction_df(raw_distinction_df):
    
    def extract_date(x):
        if x != np.nan:
            return pd.to_datetime(x.split("T")[0], format="%Y-%m-%d", errors='coerce')
        else:
            return x
        
    # Drop error on line 1176
    temp_df = raw_distinction_df.drop([1176])
    
    # Drop unnecessary columns
    temp_df = temp_df.drop(columns=['person', 'distinction'])
    
    # Rename columns
    temp_df = temp_df.rename(
        columns={
            'personLabel': 'nomComplet', 
            'distinctionLabel': 'distinction'
        }
    )
    
    # Drop nobodu
    temp_df = temp_df.dropna(subset=['nomComplet'])

    # Extract date
    temp_df['date'] = temp_df['date'].apply(lambda x : extract_date(x))
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    
    # Extract year
    temp_df['annee'] = temp_df['date'].apply(lambda x: x.year)
    temp_df['annee'] = temp_df['annee'].astype('Int32')

    return temp_df

In [230]:
raw_distinction_df = backend_load_distinction()

In [231]:
#raw_distinction_df.head()

In [232]:
distinction_df = clean_distinction_df(raw_distinction_df)

             nomComplet                                        distinction  \
0  Robert Marcel Lepage                                   Prix René-Jodoin   
1       Jean-Marie Drot                 commandeur des Arts et des Lettres   
2       Jean-Marie Drot                                   concours général   
3      François Damiens                  chevalier des Arts et des Lettres   
4         Nicole Garcia  César de la meilleure actrice dans un second rôle   

                   date  
0  2018-01-01T00:00:00Z  
1  2013-01-17T00:00:00Z  
2  1946-01-01T00:00:00Z  
3  2011-01-01T00:00:00Z  
4  1980-01-01T00:00:00Z  


In [233]:
distinction_df.head()

,nomComplet,distinction,date,annee
0,Robert Marcel Lepage,Prix René-Jodoin,2018-01-01,2018
1,Jean-Marie Drot,commandeur des Arts et des Lettres,2013-01-17,2013
2,Jean-Marie Drot,concours général,1946-01-01,1946
3,François Damiens,chevalier des Arts et des Lettres,2011-01-01,2011
4,Nicole Garcia,César de la meilleure actrice dans un second rôle,1980-01-01,1980


In [111]:
distinction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2733 entries, 0 to 2733
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   nomComplet   2733 non-null   object        
 1   distinction  2733 non-null   object        
 2   date         2731 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 85.4+ KB


In [110]:
type(raw_distinction_df.loc[0, 'date'])

str

Les deux principales données qui seront utilisées sont df_distinc et df_count

In [19]:
# Load dataframe
df_distinc = load_distinction_data()
df_distinc.head()


,person,distinction,date
0,Robert Marcel Lepage,Prix René-Jodoin,2018
1,Jean-Marie Drot,commandeur des Arts et des Lettres,2013
2,Jean-Marie Drot,concours général,1946
3,François Damiens,chevalier des Arts et des Lettres,2011
4,Nicole Garcia,César de la meilleure actrice dans un second rôle,1980


In [20]:
# Get counts of dinstinctions per person
df_count = df_distinc.groupby("person")["date"].count()

# Only consider people with more than 5 distinctions
df_count = df_count.loc[df_count >= 5]
df_count.head()

person
Alain Corneau        6
Alan Arkin          12
Alanis Obomsawin    18
Albert Uderzo        8
Alec Baldwin        15
Name: date, dtype: int64

### Dash jupyter app

In [21]:
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State


In [22]:
def compute_distinc_histogram(df_count, min_count, max_count):
    '''
    Compute the height in the scatter plot (histogram). The height
    does not encode a data per-say but is used to distinguish multiple
    persons who have the same number of distinctions.
    '''
    # Compute the height of each person in the scatter plot (custom histogram)
    df_height = pd.Series(np.zeros((df_count.size, )), index = df_count.index)

    for c in range(min_count, max_count + 1):
        # Get positions of people with "c" distinctions
        bool_idx = df_count == c
        if not bool_idx.sum() == 0:
            df_height[bool_idx] = np.arange(bool_idx.sum())

    return df_height
        

In [23]:
def get_histogram(df_count):
    '''
    Return the scatter plot (custom histogram) that represents the distribution of
    distinctions.
    '''
    min_count = df_count.min()
    max_count = df_count.max()
    df_height = compute_distinc_histogram(df_count, min_count, max_count)

    # Plot results
    fig = go.Figure(
            go.Scatter(
                mode = 'markers',
                marker_symbol = "square", 
                marker_size = 10,
                marker_color = "#B3DEC1",
                marker_line_width = 2,
                marker_line_color = "black",
                x = df_count,
                y = df_height,
                hovertemplate =
                '<br><b>Personne </b>: %{text}<br>'+
                '<b>Distinctions </b> : %{x}  <extra></extra>',
                text = df_count.index,
                showlegend = False 
                ),
            go.Layout(
                title_text = "Distribution des nombres de distinctions",
                title_x = 0.5,
                width=1000,
                height=700
                )
            )           
    fig.update_xaxes(range=[min_count - 1, max_count + 1], dtick = 1, title = "Nombre de distinctions")
    fig.update_yaxes(range=[-1, df_height.max() + 1])
    return fig


In [24]:
def get_empty_table(df_distinc):
    fig = go.Figure(
            data=[go.Table(
                    columnwidth = [600,100],
                    header=dict(values=list(df_distinc.columns[1:]),
                                fill_color="#B3DEC1",
                                align='center'),
            )],
            layout=go.Layout(
                    width=700,
                    height=700
            )
        )
                
    return fig

In [25]:
def get_filled_table(df_distinc, name):
    one_actor = df_distinc[df_distinc["person"] == name].sort_values("date", ascending = True)

    fig = go.Figure(    
            data=[go.Table(
                    columnwidth = [600,100],
                    header=dict(values=list(df_distinc.columns[1:]),
                                fill_color="#B3DEC1",
                                align='center'),
                    cells=dict(values=[one_actor["distinction"], one_actor["date"]],
                               fill_color='#E5FCF5',
                               align=['left','center'])
            )],
            layout=go.Layout(
                    title_text = f"Distinctions de {name}",
                    title_x = 0.5,
                    width=700,
                    height=700
            )
        )
         
    return fig

In [26]:
external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


In [27]:
app.layout = html.Div(className='content', children=[
    html.Header(children=[
        html.H1('Graphique des distinctions'),
    ]),
    html.Main(className='viz-container', children=[
        dcc.Graph(
            id='histogram',
            className='graph',
            figure=get_histogram(df_count),
            style={'display': 'inline-block'},
            config=dict(
                scrollZoom=False,
                showTips=False,
                showAxisDragHandles=False,
                doubleClick=False,
                displayModeBar=False
            )
        ),
        dcc.Graph(
            id='table',
            className='graph',
            figure=get_empty_table(df_distinc),
            style={'display': 'inline-block'},
            config=dict(
                scrollZoom=False,
                showTips=False,
                showAxisDragHandles=False,
                doubleClick=False,
                displayModeBar=False
            )
        )
    ])
])


In [28]:
@app.callback(
    Output('table', 'figure'),
    [Input('histogram', 'clickData')]
)
def histogram_clicked(click_data):
    if click_data is None:
        return get_empty_table(df_distinc)
    else:
        return get_filled_table(df_distinc, click_data['points'][0]['text'])

In [29]:
app.run_server(mode='jupyterlab', port=8060, dev_tools_ui=True, #debug=True, 
               dev_tools_hot_reload =True, threaded=True)

In [234]:
print("hello world")

hello world
